### 작업 요약 및 작업 진행사항 요약

#### 📸 작업 요약: 사진 파일명 표준화

##### 1. 작업 목적
`photo/` 폴더 내의 날짜별 하위 폴더(예: `202212_`, `202301_` 등)에 저장된 이미지 파일들의 이름을 **촬영 날짜** 기준으로 `YYYYMMDD_HHmmss` 형식에 맞게 일괄 변경하여 파일 관리의 일관성 및 효율성을 확보합니다.

##### 2. 작업 대상
* **원본 경로:** 도커 컨테이너 내부의 `/data/photo` (로컬 `D:\conda\photo`와 연결됨)
* **대상 파일:** `jpg`, `jpeg`, `png`, `heic` 등 이미지 파일

##### 3. 핵심 로직
* **날짜 추출 (1순위):** 이미지의 **EXIF 메타데이터**에서 '촬영 원본 날짜' (`DateTimeOriginal`)를 추출하여 파일명으로 사용합니다.
* **날짜 추출 (2순위):** EXIF 정보가 없는 파일(예: PNG, 스크린샷, 카카오톡 캡처 등)은 **'파일 최종 수정 시간'** (`mtime`)을 기준으로 파일명을 생성합니다.
* **중복 처리:** 동일한 시간에 촬영된 파일(예: 버스트 샷)로 인해 파일명 충돌이 발생할 경우, `_1`, `_2`와 같이 숫자 접미사를 붙여 원본 파일이 덮어씌워지는 것을 방지합니다.

---

#### 📊 작업 진행사항 요약

##### 1. 진행 방식
* 제공된 Python 스크립트(`.ipynb` 셀)를 사용하여 연도별로 작업을 분리하여 순차적으로 실행.
* 스크립트 내 `dirnames[:] = [d for d in dirnames if d.startswith('20XX')]` 라인의 `20XX` 값을 변경하며 진행함.

##### 2. 현재 상태 (2025-11-02 기준)

* **[완료]** `2022`년도 폴더 (`startswith('2022')`) 전체 파일명 변경 작업 완료. (2025.11.02.)
* **[완료]** `2023`년도 폴더 (`startswith('2023')`) 전체 파일명 변경 작업 완료. (2025.11.02.)
* **[완료]** `2024`년도 폴더 (`startswith('2024')`) 전체 파일명 변경 작업 완료. (2025.11.02.)
* **[완료]** `2025`년도 폴더 (`startswith('2024')`) 전체 파일명 변경 작업 완료. (2025.11.04.)

---

#### 🚀 향후 작업 진행 
* 스크립트 필터 값을 `folder_start = '2025'`로 변경하여 남은 `2025`년도 폴더에 대한 파일명 표준화 작업을 실행할 예정


In [1]:
# !conda install -y -c conda-forge libheif libstdcxx-ng pymediainfo

In [ ]:
import os
import re
from datetime import datetime
import shutil

# 1. Pillow (이미지)
from PIL import Image
import pillow_heif  # HEIC/HEIF 활성화

# 2. MediaInfo (비디오)
from pymediainfo import MediaInfo

# --- 1. 설정 ---

# [중요] Docker 컨테이너 내부의 작업 경로입니다.
# 우리가 -v "//d/conda:/data" 로 연결했습니다.
base_dir = "/data/photo"

# 'DateTimeOriginal' (사진 촬영 원본 시간) 태그의 고유 ID입니다.
exif_date_tag = 36867

In [3]:
# --- 2. 날짜/시간 포맷 함수 ---

def format_datetime_str(dt_str):
    """EXIF 날짜 형식 ('2023:10:27 15:30:00')을 파싱하여 포맷에 맞게 변환"""
    try:
        dt = datetime.strptime(dt_str, '%Y:%m:%d %H:%M:%S')
        return dt.strftime('%Y%m%d_%H%M%S')
    except ValueError:
        return None

def format_timestamp(ts):
    """파일 시스템 시간 (Timestamp)을 포맷에 맞게 변환 (Fallback 용)"""
    dt = datetime.fromtimestamp(ts)
    return dt.strftime('%Y%m%d_%H%M%S')


In [ ]:
# --- 3. 재귀적으로 파일 탐색 및 이름 변경 ---

print(f"'{base_dir}'에서 파일 이름 변경 작업을 시작합니다...")
print("-" * 40)

total_files = 0
renamed_files = 0
failed_files = 0

# [최적화]
# 'photo' 폴더(base_dir)에 처음 진입했을 때,
# 하위 폴더 목록(dirnames)을 '2022'으로 시작하는 폴더로 제한합니다.
# 이렇게 하면 '202212_' 같은 폴더만 탐색하게 됩니다.
folder_start = '2025'

# os.walk로 모든 하위 폴더 탐색
print(f"'{base_dir}'에서 파일 이름 변경 작업을 시작합니다...")
print("-" * 40)

total_files = 0
renamed_files = 0
failed_files = 0

# os.walk로 모든 하위 폴더 탐색
for dirpath, dirnames, filenames in os.walk(base_dir):

    # [최적화] (folder_start 설정 시)
    if dirpath == base_dir and folder_start:
        original_dirnames = list(dirnames)
        dirnames[:] = [d for d in dirnames if d.startswith(folder_start)]
        print(f"탐색 대상 폴더들: {dirnames}")
        ignored_dirs = [d for d in original_dirnames if d not in dirnames]
        if ignored_dirs:
            print(f"(무시된 폴더: {ignored_dirs})")
        print("-" * 40)

    for filename in filenames:
        total_files += 1
        original_filepath = os.path.join(dirpath, filename)

        file_base, file_ext = os.path.splitext(filename)
        file_ext_lower = file_ext.lower()

        # ⭐️ [수정] .mov 확장자 추가
        if file_ext_lower not in ['.jpg', '.jpeg', '.png', '.tif', '.tiff', '.heic', '.mov']:
            continue

        new_name_base = None
        date_source = None

        # --- 1순위: 임베디드 '미디어 생성 날짜' 시도 ---
        try:
            # 1-A: 이미지 파일 (Pillow)
            if file_ext_lower in ['.jpg', '.jpeg', '.png', '.tif', '.tiff', '.heic']:
                with Image.open(original_filepath) as img:
                    exif_data = img.getexif() # 👈 HEIC 호환
                    if exif_data and exif_date_tag in exif_data:
                        exif_str = exif_data[exif_date_tag]
                        new_name_base = format_datetime_str(exif_str)
                        date_source = "EXIF"

            # ⭐️ 1-B: [신규] 비디오 파일 (pymediainfo)
            elif file_ext_lower == '.mov':
                media_info = MediaInfo.parse(original_filepath)
                creation_date_str = None

                # 'General' 트랙에서 'Encoded date' 또는 'Tagged date'를 찾음
                for track in media_info.tracks:
                    if track.track_type == 'General':
                        creation_date_str = track.encoded_date or track.tagged_date
                        if creation_date_str:
                            break

                if creation_date_str:
                    # 'pymediainfo'는 'UTC 2025-04-28 14:21:15' 형식으로 반환
                    if creation_date_str.startswith('UTC '):
                        date_str_only = creation_date_str[4:]
                    else:
                        date_str_only = creation_date_str

                    # 'YYYY-MM-DD HH:MM:SS' 형식을 'YYYY:MM:DD HH:MM:SS'로 변환
                    dt_obj = datetime.strptime(date_str_only, '%Y-%m-%d %H:%M:%S')
                    formatted_for_helper = dt_obj.strftime('%Y:%m:%d %H:%M:%S')

                    # 헬퍼 함수 사용
                    new_name_base = format_datetime_str(formatted_for_helper)
                    date_source = "MediaInfo"

        except Exception as e:
            # EXIF 또는 MediaInfo 읽기 실패
            # print(f"  [정보] {filename} 1순위 메타데이터 읽기 실패: {e}")
            pass

        # --- 2순위: 1순위가 실패하면 파일 '수정 시간' 사용 ---
        if new_name_base is None:
            try:
                file_time = os.path.getmtime(original_filepath)
                new_name_base = format_timestamp(file_time)
                date_source = "FileModTime"
            except Exception as e:
                print(f"  [오류] {filename}의 시간 정보를 읽지 못했습니다: {e}")
                failed_files += 1
                continue

        # --- 새 파일 이름 생성 및 충돌 처리 ---
        if new_name_base is None:
            print(f"  [오류] {filename}에서 날짜 정보를 추출하지 못했습니다.")
            failed_files += 1
            continue

        # [수정] 확장자를 원본의 대소문자(file_ext) 그대로 사용
        new_filename = f"{new_name_base}{file_ext}"
        new_filepath = os.path.join(dirpath, new_filename)

        if original_filepath == new_filepath:
            # print(f"  [유지] {filename} (이미 올바른 이름)")
            continue

        counter = 1
        while os.path.exists(new_filepath):
            new_filename = f"{new_name_base}_{counter}{file_ext}"
            new_filepath = os.path.join(dirpath, new_filename)
            counter += 1

        # --- 파일 이름 변경 실행 ---
        try:
            os.rename(original_filepath, new_filepath)
            print(f"  [변경] {filename} -> {new_filename} (Source: {date_source})")
            renamed_files += 1
        except Exception as e:
            print(f"  [오류] {filename} 이름 변경 실패: {e}")
            failed_files += 1

print("\n" + "-" * 40)
print("--- 📸 작업 완료 ---")
print(f"총 스캔한 파일 (이미지/영상): {total_files} (필터링 된 기준)")
print(f"이름 변경된 파일: {renamed_files}")
print(f"실패/스킵된 파일: {failed_files}")

'/data/photo'에서 파일 이름 변경 작업을 시작합니다...
----------------------------------------
'/data/photo'에서 파일 이름 변경 작업을 시작합니다...
----------------------------------------
탐색 대상 폴더들: ['202504__', '202505__', '202506__', '202507__', '202508__', '202509__', '202510__']
----------------------------------------
  [변경] 20250405_135106_2.MOV -> 20250405_135106_1.MOV (Source: MediaInfo)
  [변경] KakaoTalk_20251108_200236864.jpg -> 20251110_005451.jpg (Source: FileModTime)
  [변경] KakaoTalk_20251108_200259157.jpg -> 20251110_005457.jpg (Source: FileModTime)
  [변경] KakaoTalk_20251108_200302002.jpg -> 20251110_005502.jpg (Source: FileModTime)
  [변경] KakaoTalk_20251108_200310320.jpg -> 20251110_005505.jpg (Source: FileModTime)
  [변경] KakaoTalk_20251108_200311642.jpg -> 20251110_005507.jpg (Source: FileModTime)
  [변경] KakaoTalk_20251108_200312241.jpg -> 20251110_005510.jpg (Source: FileModTime)
  [변경] KakaoTalk_20251108_200312752.jpg -> 20251110_005512.jpg (Source: FileModTime)
  [변경] KakaoTalk_20251108_200313230.